In [1]:
import pandas as pd
import os
import time
from datetime import datetime
from matplotlib.dates import date2num
import sqlite3

In [119]:
# Create NEW SQL database
connect = sqlite3.connect('Yahoo_sqlite.db')
cursor = connect.cursor()


PATH = '/Users/insuyu/GitHub/historical.data/yahoo.csv/'
#PATH = '/Users/insuyu/GitHub/python/11. Sqlite3/'


# For All Files in PATH
nFiles = 0
for filename in os.listdir(PATH):
    # Find only *.csv files
    if filename.endswith(".csv"):
        print ("[%d] %s" % (nFiles, filename))
        # Load CVS to Database
        DB = pd.read_csv(PATH+filename)
    
        # Generate Moving Average Data
        DB['MA20'] = pd.stats.moments.rolling_mean(DB['Adj Close'], 20)
        DB['MA30'] = pd.stats.moments.rolling_mean(DB['Adj Close'], 30)
        #DB['MA30'] = pd.stats.moments.rolling_mean(DB['Adj Close'], 30)
        #DB['MA60'] = pd.stats.moments.rolling_mean(DB['Adj Close'], 60)
        #DB['MA120'] = pd.stats.moments.rolling_mean(DB['Adj Close'], 120)
        DB['MA200']     = pd.stats.moments.rolling_mean(DB['Adj Close'], 200)
        DB['MA240']     = pd.stats.moments.rolling_mean(DB['Adj Close'], 240)
        DB['DateStamp'] = [ date2num(datetime.strptime(date,"%Y-%m-%d")) for date in DB['Date'] ]
        
        ###########################################################################
        Code = filename[:-4] # Trading Code
        
        # Create a New Table for Code
        # DateStamp(REAL) | Date(TEXT) | Open(REAL) | High(REAL) | Low(REAL) 
        #    | Close(REAL) | Volume(REAL) | AdjClose(REAL) | MA20(REAL) | MA30(REAL) | MA200(REAL) | MA240 (REAL)
        command = "CREATE TABLE KS%s(DateStamp INT, Date TEXT, Open REAL, High REAL, Low REAL,  \
                        Close REAL, Volume INT, AdjClose REAL, MA20 REAL, MA30 REAL, MA200 REAL, MA240 REAL)" % Code
        print(command)
        cursor.execute(command)
        
        command = "INSERT INTO KS%s(DateStamp, Date, Open, High, Low, Close, Volume, AdjClose, MA20, MA30, MA200, MA240) \
        VALUES (?,?,?,?,?,?,?,?,?,?,?,?)" % Code
        print(command)
        
        for i in range(len(DB)):
            DateStamp = DB['DateStamp'][i]
            Date = DB['Date'][i]
            Open = DB['Open'][i]
            High = DB['High'][i]
            Low = DB['Low'][i] 
            Close = DB['Close'][i]
            Volume = DB['Volume'][i]
            AdjClose = DB['Adj Close'][i]
            MA20 = DB['MA20'][i]
            MA30 = DB['MA30'][i]
            MA200 = DB['MA200'][i]
            MA240 = DB['MA240'][i]
            
            cursor.execute(command,(DateStamp, Date, Open, High, Low, Close, Volume, AdjClose, MA20, MA30, MA200, MA240) )
            connect.commit()
        nFiles += 1
            


[0] 000020.csv
CREATE TABLE KS000020(DateStamp INT, Date TEXT, Open REAL, High REAL, Low REAL,                          Close REAL, Volume REAL, AdjClose REAL, MA20 REAL, MA30 REAL, MA200 REAL, MA240 REAL)
INSERT INTO KS000020(DateStamp, Date, Open, High, Low, Close, Volume, AdjClose, MA20, MA30, MA200, MA240)         VALUES (?,?,?,?,?,?,?,?,?,?,?,?)


KeyError: 'MA20'

### Convert a string "2015-8-1" to 73468.0

In [29]:
PATH = '/Users/insuyu/GitHub/python/12. CSV2SQL/'
# conversion "%Y-%m-%d" string to date number
DB = pd.read_csv(PATH+'003490.csv')
date = DB['Date'][20]
date2num(datetime.strptime(date,"%Y-%m-%d"))

730151.0

### Add DateStamp to DB

In [31]:
DB['MA20'] = pd.stats.moments.rolling_mean(DB['Adj Close'], 20)
DB['DateStamp'] = [ date2num(datetime.strptime(date,"%Y-%m-%d")) for date in DB['Date'] ]
DB

,Date,Open,High,Low,Close,Volume,Adj Close,MA30,MA60,MA120,MA240,MA20,DateStamp
0,2000-01-04,10700.02,11800.04,10700.02,11800.04,1810000,14564.59,NaN,NaN,NaN,NaN,NaN,730123
1,2000-01-05,11599.98,13099.96,11099.98,13099.96,2130200,16169.06,NaN,NaN,NaN,NaN,NaN,730124
2,2000-01-06,13000.01,13000.01,11299.97,11599.98,1499700,14317.66,NaN,NaN,NaN,NaN,NaN,730125
3,2000-01-07,12000.02,12250.02,11400.00,12000.02,1283400,14811.42,NaN,NaN,NaN,NaN,NaN,730126
4,2000-01-10,11800.04,12200.00,11649.99,11750.02,840700,14502.86,NaN,NaN,NaN,NaN,NaN,730129
5,2000-01-11,11599.98,12099.97,11549.96,11800.04,994200,14564.59,NaN,NaN,NaN,NaN,NaN,730130
6,2000-01-12,11599.98,11700.01,11099.98,11099.98,710800,13700.52,NaN,NaN,NaN,NaN,NaN,730131
7,2000-01-13,10900.00,11500.03,10900.00,11450.01,672100,14132.56,NaN,NaN,NaN,NaN,NaN,730132
8,2000-01-14,11899.99,11899.99,11049.97,11150.00,617800,13762.26,NaN,NaN,NaN,NaN,NaN,730133
9,2000-01-17,10900.00,11000.03,10299.98,10750.04,775100,13268.59,NaN,NaN,NaN,NaN,NaN,730136


## Access SQLite data

In [58]:
connect = sqlite3.connect('Yahoo_sqlite.db')
cursor = connect.cursor()

In [59]:
sql = "SELECT * FROM YAHOO WHERE Date =?"

for row in cursor.execute(sql, [('2015-06-15')]):
    print (row)

('003490', 735764, '2015-06-15', 36950.0, 36950.0, 35150.0, 35200.0, 817100, 35200.0, 40119.999999999956, 41351.66666666669, 43575.00000000003, 42138.124999999905)
('005930', 735764, '2015-06-15', 1255000.0, 1274000.0, 1255000.0, 1270000.0, 161300, 1270000.0, 1312749.9999999988, 1323700.0000000005, 1307059.9780500038, 1299703.0276250048)


In [60]:
for row in cursor.execute("SELECT * FROM YAHOO WHERE Date > '2015-07-01' "):
    print (row)

('003490', 735781, '2015-07-02', 42500.0, 43850.0, 41900.0, 42900.0, 1340700, 42900.0, 38622.499999999956, 39331.66666666669, 43631.25000000003, 42362.91666666658)
('003490', 735782, '2015-07-03', 42950.0, 43450.0, 41450.0, 41500.0, 583100, 41500.0, 38747.499999999956, 39250.00000000002, 43664.00000000003, 42391.45833333324)
('003490', 735785, '2015-07-06', 40800.0, 42600.0, 40700.0, 41200.0, 587800, 41200.0, 38862.499999999956, 39158.33333333336, 43691.75000000003, 42420.624999999905)
('003490', 735786, '2015-07-07', 42600.0, 44500.0, 42050.0, 43550.0, 1506500, 43550.0, 39052.499999999956, 39206.66666666669, 43729.50000000003, 42462.29166666658)
('003490', 735787, '2015-07-08', 43550.0, 43900.0, 41900.0, 42000.0, 601500, 42000.0, 39249.999999999956, 39231.66666666669, 43765.00000000003, 42500.624999999905)
('003490', 735788, '2015-07-09', 41000.0, 41500.0, 38950.0, 39700.0, 1608600, 39700.0, 39329.999999999956, 39186.66666666669, 43787.75000000003, 42526.874999999905)
('003490', 73578

In [63]:
for row in cursor.execute("SELECT * FROM YAHOO WHERE Date > '2015-07-01' AND Date < '2015-07-08'"):
    print (row)

('003490', 735781, '2015-07-02', 42500.0, 43850.0, 41900.0, 42900.0, 1340700, 42900.0, 38622.499999999956, 39331.66666666669, 43631.25000000003, 42362.91666666658)
('003490', 735782, '2015-07-03', 42950.0, 43450.0, 41450.0, 41500.0, 583100, 41500.0, 38747.499999999956, 39250.00000000002, 43664.00000000003, 42391.45833333324)
('003490', 735785, '2015-07-06', 40800.0, 42600.0, 40700.0, 41200.0, 587800, 41200.0, 38862.499999999956, 39158.33333333336, 43691.75000000003, 42420.624999999905)
('003490', 735786, '2015-07-07', 42600.0, 44500.0, 42050.0, 43550.0, 1506500, 43550.0, 39052.499999999956, 39206.66666666669, 43729.50000000003, 42462.29166666658)
('005930', 735781, '2015-07-02', 1286000.0, 1304000.0, 1285000.0, 1299000.0, 164400, 1299000.0, 1281849.9999999988, 1294433.3333333337, 1313837.9104500038, 1296961.581041671)
('005930', 735782, '2015-07-03', 1287000.0, 1294000.0, 1267000.0, 1268000.0, 142600, 1268000.0, 1278199.9999999988, 1291700.0000000005, 1314338.3671500038, 1296939.225250

In [62]:
for row in cursor.execute("SELECT * FROM YAHOO WHERE DateStamp > 735788"):
    print (row)

('003490', 735789, '2015-07-10', 40100.0, 40150.0, 39500.0, 39750.0, 512700, 39750.0, 39442.499999999956, 39163.33333333336, 43810.75000000003, 42555.20833333324)
('003490', 735792, '2015-07-13', 40050.0, 40550.0, 39750.0, 40450.0, 452200, 40450.0, 39704.999999999956, 39216.66666666669, 43837.75000000003, 42585.624999999905)
('003490', 735793, '2015-07-14', 40700.0, 40800.0, 39400.0, 39450.0, 657700, 39450.0, 39889.999999999956, 39245.00000000002, 43859.50000000003, 42612.499999999905)
('003490', 735794, '2015-07-15', 40450.0, 40500.0, 39400.0, 39450.0, 886900, 39450.0, 40082.499999999956, 39270.00000000002, 43880.75000000003, 42637.08333333324)
('003490', 735795, '2015-07-16', 39000.0, 40300.0, 38700.0, 39450.0, 9033400, 39450.0, 40269.999999999956, 39298.33333333336, 43902.00000000003, 42661.874999999905)
('003490', 735796, '2015-07-17', 39550.0, 40000.0, 38250.0, 39000.0, 1149500, 39000.0, 40327.499999999956, 39298.33333333336, 43919.50000000003, 42684.79166666658)
('003490', 735799